In [84]:
import torch
import torch.nn as nn
from torch_scatter import scatter_add
import torch.optim as optim

import numpy as np 
import pandas as pd
import time
import os

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

from torch_geometric.data import Data, Dataset#,DataLoader
from torch_geometric.loader import DataLoader
from torch import Tensor

import torch.nn.functional as F
import torch_geometric.transforms as Tr
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import to_networkx
from torch.nn import Sequential as Seq, Linear, ReLU, Sigmoid
from torch.optim.lr_scheduler import StepLR
from collections import namedtuple

In [104]:
class GraphDataset(Dataset):
    def __init__(self, graph_files, file_name, transform=None, pre_transform=None):
        super(GraphDataset, self).__init__()

        self.graph_files = graph_files
        self.file_name = file_name

    @property
    def raw_file_names(self):
        return self.graph_files

    @property
    def processed_file_names(self):
        return []

    def get(self, idx):
        data = torch.load(f'/users/santoshp/trackml_data/mockdata/{self.file_name}' + f'event{idx}.pyg')
        print(f"Loaded data for graph {idx}:", data)
        return data

    def len(self):
        return len(self.graph_files)


In [105]:
home_dir = "../"
test = 'testset/'
train = 'trainset/'
val = 'valset/'
indir = '/users/santoshp/trackml_data/mockdata/'
    
graph_files_test = np.array(os.listdir(indir + test))
graph_files_test = [os.path.join(indir,file)
                           for file in graph_files_test]
graph_files_train = np.array(os.listdir(indir+train))
graph_files_train = [os.path.join(indir, file)
                            for file in graph_files_train]
graph_files_val = np.array(os.listdir(indir+val))
graph_files_val = [os.path.join(indir,file)
                            for file in graph_files_val]

In [106]:
 graph_files_train

['/users/santoshp/trackml_data/mockdata/event0.pyg']

In [107]:
params = {'batch_size': 8, 'shuffle': True, 'num_workers': 0}
train_set = GraphDataset(graph_files_train,train)
train_loader = DataLoader(train_set,**params)  #batches join graphs instead of splitting them therefore more than train set batches will make 1 batch only 
test_set = GraphDataset(graph_files_test, test)
test_loader = DataLoader(test_set, **params)
val_set = GraphDataset(graph_files_val, val)
val_loader = DataLoader(val_set, **params)

In [129]:
for i in range(1):
  print(test_set.get(i))
for batch_idx, data in enumerate(test_loader):
   print(data)

Loaded data for graph 0: DataBatch(leta=[14524], cell_count=[14524], z=[14524], phi=[14524], module_index=[14524], lphi=[14524], ly=[14524], hit_id=[14524], r=[14524], lz=[14524], cell_val=[14524], y=[187], gphi=[14524], weight=[14524], lx=[14524], eta=[14524], geta=[14524], x=[14524], region=[14524], track_edges=[2, 69], pt=[69], radius=[69], particle_id=[69], nhits=[69], config=[2], event_id=[1], num_nodes=14524, batch=[14524], edge_index=[2, 187], truth_map=[69], weights=[187], scores=[187])
DataBatch(leta=[14524], cell_count=[14524], z=[14524], phi=[14524], module_index=[14524], lphi=[14524], ly=[14524], hit_id=[14524], r=[14524], lz=[14524], cell_val=[14524], y=[187], gphi=[14524], weight=[14524], lx=[14524], eta=[14524], geta=[14524], x=[14524], region=[14524], track_edges=[2, 69], pt=[69], radius=[69], particle_id=[69], nhits=[69], config=[2], event_id=[1], num_nodes=14524, batch=[14524], edge_index=[2, 187], truth_map=[69], weights=[187], scores=[187])
Loaded data for graph 0: 

In [109]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [110]:
print(train_set)

GraphDataset()


In [111]:
class RelationalModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(RelationalModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, m):
        return self.layers(m)

class ObjectModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ObjectModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, C):
        return self.layers(C)


class InteractionNetwork(MessagePassing):
    def __init__(self, node_f_size, edge_attr_size,message_out, update_out, hidden_size):
        super(InteractionNetwork, self).__init__(aggr='add', 
                                                 flow='source_to_target')
        self.R1 = RelationalModel(2*node_f_size + edge_attr_size, message_out, hidden_size)    # 19 is the node_features * 2 + edge atributes output 4 
        self.O = ObjectModel(node_f_size + message_out, update_out, hidden_size)    # 10 is node features + output R1
        self.R2 = RelationalModel(2*update_out + message_out , 1, hidden_size)  #10 is from 2* output O + output R1(from the concat) 
        self.E: Tensor = Tensor()

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]],
                        self.E], dim=1)
        return torch.sigmoid(self.R2(m2))
        
    def message(self, x_i, x_j, edge_attr):
        m1 = torch.cat([x_i, x_j, edge_attr], dim=1)
        self.E = self.R1(m1)
        return self.E

    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.O(c) 

In [112]:
print(train_set.get(0))

Loaded data for graph 0: DataBatch(leta=[14524], cell_count=[14524], z=[14524], phi=[14524], module_index=[14524], lphi=[14524], ly=[14524], hit_id=[14524], r=[14524], lz=[14524], cell_val=[14524], y=[187], gphi=[14524], weight=[14524], lx=[14524], eta=[14524], geta=[14524], x=[14524], region=[14524], track_edges=[2, 69], pt=[69], radius=[69], particle_id=[69], nhits=[69], config=[2], event_id=[1], num_nodes=14524, batch=[14524], edge_index=[2, 187], truth_map=[69], weights=[187], scores=[187])
DataBatch(leta=[14524], cell_count=[14524], z=[14524], phi=[14524], module_index=[14524], lphi=[14524], ly=[14524], hit_id=[14524], r=[14524], lz=[14524], cell_val=[14524], y=[187], gphi=[14524], weight=[14524], lx=[14524], eta=[14524], geta=[14524], x=[14524], region=[14524], track_edges=[2, 69], pt=[69], radius=[69], particle_id=[69], nhits=[69], config=[2], event_id=[1], num_nodes=14524, batch=[14524], edge_index=[2, 187], truth_map=[69], weights=[187], scores=[187])


In [118]:
# Define the size of layers in the neural network
hidden_l_size = 12    # Tunable parameter
message_out = 4       # Tunable parameter
update_out = 3        # Tunable parameter

# Get the size of edge attributes and node features from the dataset
data = train_set.get(0)

# Check if edge_attr is not None and has a shape before accessing its shape
if data.edge_attr is not None and hasattr(data.edge_attr, 'shape') and len(data.edge_attr.shape) > 1:
    edge_attr_size = data.edge_attr.shape[1]
else:
    # Handle the case when edge_attr is None or doesn't have the expected shape
    edge_attr_size = 0  # or any other appropriate value

# Similarly, check for the 'x' attribute
if data.x is not None and hasattr(data.x, 'shape') and len(data.x.shape) > 1:
    node_f_size = data.x.shape[1]
else:
    # Handle the case when 'x' is None or doesn't have the expected shape
    node_f_size = 0  # or any other appropriate value

# Initialize the model
model = InteractionNetwork(node_f_size=node_f_size, edge_attr_size=edge_attr_size, message_out=message_out, update_out=update_out, hidden_size=hidden_l_size).to(device)


Loaded data for graph 0: DataBatch(leta=[14524], cell_count=[14524], z=[14524], phi=[14524], module_index=[14524], lphi=[14524], ly=[14524], hit_id=[14524], r=[14524], lz=[14524], cell_val=[14524], y=[187], gphi=[14524], weight=[14524], lx=[14524], eta=[14524], geta=[14524], x=[14524], region=[14524], track_edges=[2, 69], pt=[69], radius=[69], particle_id=[69], nhits=[69], config=[2], event_id=[1], num_nodes=14524, batch=[14524], edge_index=[2, 187], truth_map=[69], weights=[187], scores=[187])


/users/santoshp/.conda/envs/gnn4itk/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [120]:
def binary_acc(y_pred, y_test,thld):
  """
  returns accuracy based on a given treshold
  """

  # true positives edges with ouput prediction bigger than thld(1) and label = 1
  TP = torch.sum((y_test==1.).squeeze() & 
                           (y_pred>thld).squeeze()).item()
  # true negatives edges with ouput prediction smaller than thld(0) and label = 0
  TN = torch.sum((y_test==0.).squeeze() & 
                           (y_pred<thld).squeeze()).item()
  # False positives edges with ouput prediction bigger than thld(1) and label = 0
  FP = torch.sum((y_test==0.).squeeze() & 
                           (y_pred>thld).squeeze()).item()
  # False negatives edges with ouput prediction smaller than thld(0) and label = 1                     
  FN = torch.sum((y_test==1.).squeeze() & 
                           (y_pred<thld).squeeze()).item() 
  #how many correct predictions are made, if FP = 0 and FN = 0 acc = 1                       
  acc = (TP+TN)/(TP+TN+FP+FN)
    
  return acc

In [121]:
total_trainable_params = sum(p.numel() for p in model.parameters())
print('total trainable params:', total_trainable_params)

total trainable params: 776


In [127]:
preccision_acc = 0.5
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    epoch_t0 = time()
    losses = []
    accs = []
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.edge_attr)
        y, output = data.y, output.squeeze(1)
        loss = F.binary_cross_entropy(output, y, reduction='mean')
        loss.backward()
        optimizer.step()
       
        # if batch_idx % 10 == 0:
        #   print(f"""Train Epoch:{epoch} [{batch_idx}/{len(train_loader.dataset)}({100. * batch_idx / len(train_loader):.0f}%)]""")
        # if batch_idx % 10 == 0:
        #     print(f"""Train Epoch:{epoch}""")
            
            # if args.dry_run:
            #     break
        losses.append(loss.item())   #this is the same as loss because there is no batches 
        accs.append(binary_acc(y_pred = output, y_test = y, thld = preccision_acc ))

    print(f"...epoch time: {time()-epoch_t0}s")
    print(f"...epoch {epoch}: mean train loss={np.mean(losses):.6f}......train acc={np.mean(accs):.6f}")
    return np.mean(losses)

In [124]:
 preccision_acc = 0.5
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    epoch_t0 = time()
    losses = []
    accs = []
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.edge_attr)
        y, output = data.y, output.squeeze(1)
        loss = F.binary_cross_entropy(output, y, reduction='mean')
        loss.backward()
        optimizer.step()
       
        # if batch_idx % 10 == 0:
        #   print(f"""Train Epoch:{epoch} [{batch_idx}/{len(train_loader.dataset)}({100. * batch_idx / len(train_loader):.0f}%)]""")
        # if batch_idx % 10 == 0:
        #     print(f"""Train Epoch:{epoch}""")
            
            # if args.dry_run:
            #     break
        losses.append(loss.item())   #this is the same as loss because there is no batches 
        accs.append(binary_acc(y_pred = output, y_test = y, thld = preccision_acc ))

    print(f"...epoch time: {time()-epoch_t0}s")
    print(f"...epoch {epoch}: mean train loss={np.mean(losses):.6f}......train acc={np.mean(accs):.6f}")
    return np.mean(losses)

def validate(model, device, val_loader):
    model.eval()
    opt_thlds, accs = [], []
    for batch_idx, data in enumerate(val_loader):
        data = data.to(device)
        output = model(data.x, data.edge_index, data.edge_attr)
        y, output = data.y, output.squeeze()
        loss = F.binary_cross_entropy(output, y, reduction='mean').item()
        
        # define optimal threshold (thld) where TPR = TNR 
        diff, opt_thld, opt_acc = 100, 0, 0
        best_tpr, best_tnr = 0, 0
        for thld in np.arange(0.001, 0.5, 0.001):
            TP = torch.sum((y==1) & (output>thld)).item()
            TN = torch.sum((y==0) & (output<thld)).item()
            FP = torch.sum((y==0) & (output>thld)).item()
            FN = torch.sum((y==1) & (output<thld)).item()
            acc = (TP+TN)/(TP+TN+FP+FN)
            TPR, TNR = TP/(TP+FN), TN/(TN+FP)
            delta = abs(TPR-TNR)
            if (delta < diff): 
                diff, opt_thld, opt_acc = delta, thld, acc

        opt_thlds.append(opt_thld)
        accs.append(opt_acc)

    print(f"...........................................val acc={np.mean(accs):.6f}")
    return np.mean(opt_thlds) 

def test(model, device, test_loader, thld=0.5):
    model.eval()
    test_t0 = time()
    losses, accs = [], []
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            data = data.to(device)
            output = model(data.x, data.edge_index, data.edge_attr)
            acc = binary_acc(y_pred = output, y_test = data.y, thld =  thld)
            loss = F.binary_cross_entropy(output.squeeze(1), data.y, 
                                          reduction='mean').item()
            accs.append(acc)
            losses.append(loss)
    
    #when batching works change acc for mean accs
    print(f"...testing time: {time()-test_t0}s")
    print(f'.............mean test loss={np.mean(losses):.6f}......test acc ={acc:.6f}\n')
    return np.mean(losses), np.mean(accs)

In [130]:
from time import time
output = {'train_loss': [], 'test_loss': [], 'test_acc': []}
SAVE_MODEL = False
for epoch in range(1, 50 + 1):
  print("---- Epoch {} ----".format(epoch))
  train_loss = train( model, device, train_loader, optimizer, epoch)
  thld = validate(model, device, val_loader)
  #print('...optimal threshold', thld)
  test_loss, test_acc = test(model, device, test_loader, thld=thld)
  scheduler.step()

---- Epoch 1 ----
Loaded data for graph 0: DataBatch(leta=[14524], cell_count=[14524], z=[14524], phi=[14524], module_index=[14524], lphi=[14524], ly=[14524], hit_id=[14524], r=[14524], lz=[14524], cell_val=[14524], y=[187], gphi=[14524], weight=[14524], lx=[14524], eta=[14524], geta=[14524], x=[14524], region=[14524], track_edges=[2, 69], pt=[69], radius=[69], particle_id=[69], nhits=[69], config=[2], event_id=[1], num_nodes=14524, batch=[14524], edge_index=[2, 187], truth_map=[69], weights=[187], scores=[187])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)